In [16]:
from langchain.document_loaders import PyPDFDirectoryLoader, TextLoader, CSVLoader
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.storage import LocalFileStore
from langchain.retrievers import BM25Retriever, EnsembleRetriever, ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor, LLMChainFilter, EmbeddingsFilter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import DeepInfra
from langchain.chains import ConversationalRetrievalChain
from langchain_core.utils.env import get_from_env
from dotenv import load_dotenv
import streamlit as st
from streamlit_chat import message
import tempfile
import os

In [17]:

# Accessing the OPENAI_API_KEY variable
openaiapikey = os.environ.get('OPENAI_API_KEY')

# Accessing the DEEPINFRA_API_TOKEN variable
deeptoken = os.environ.get('DEEPINFRA_API_TOKEN')

In [18]:
compressor_llm = DeepInfra(model_id="mistralai/Mistral-7B-Instruct-v0.1")
compressor_llm.model_kwargs = {
    "temperature": 0.1,
    "repetition_penalty": 1.2,
    "max_new_tokens": 500,
    "top_p": 0.90,
}

In [19]:
embeddings = OpenAIEmbeddings(model='text-embedding-ada-002')
# Helper function for printing docs
def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))
    

In [20]:
st.title("hidfddddfdfdf")

2023-12-30 17:10:52.638 
  command:

    streamlit run C:\Users\rshoc\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()

In [5]:
loader = PyPDFDirectoryLoader("Documents")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap  = 300,
    length_function = len,
    is_separator_regex = False,
)
documents  = text_splitter.split_documents(docs)


In [6]:
%%time
# Define the path for the FAISS vector store
FAISS_DB_PATH = 'vectorStore/cse/semester8'


if os.path.exists(FAISS_DB_PATH):
   db = FAISS.load_local(FAISS_DB_PATH, embeddings)
   print(f"vectorDB already present in dir : {FAISS_DB_PATH}")

else:
    FAISS_DB_PATH = 'vectorStore/cse/semester8'
    db = FAISS.from_documents(documents, embeddings)
    db.save_local(FAISS_DB_PATH)


    print(f"vectorDB created in dir : {FAISS_DB_PATH}")


vectorDB already present in dir : vectorStore/cse/semester8
CPU times: total: 609 ms
Wall time: 1.08 s


In [7]:
faiss_retriever = db.as_retriever(search_kwargs={"k": 4})
" k   define no of top relevant documents to be retrieved"

bm25_retriever = BM25Retriever.from_documents(documents)
bm25_retriever.k = 4


# initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever], weights=[0.5, 0.5]
)

In [8]:
user_query =  '''
 List out the key elements or components involved in parallel computing, and how do these elements work together to execute tasks simultaneously and improve the overall performance and efficiency of computing systems?
''' ## make this user query to inout field in streamlit application to get user query

docs = ensemble_retriever.get_relevant_documents(user_query)


In [9]:

if (input("enter quality mode : high or average")=='high'):   ##this must be made into a mode button in the streamlit side bar (retrieval processing -> quality or standard)
    compressor = LLMChainExtractor.from_llm(compressor_llm)
else:
    compressor = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.80)
    print("using similarity_threshold")
    
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=ensemble_retriever)

using similarity_threshold


In [10]:
compressed_docs = compression_retriever.get_relevant_documents(user_query)
pretty_print_docs(compressed_docs)

Document 1:

Review questions............................................................................................................. 27
CHAPTER 2 Principles of Parallel and Distributed Computing ............................ 29
2.1Eras of computing..................................................................................................... 29
2.2Parallel vs. distributed computing............................................................................ 29
2.3Elements of parallel computing ............................................................................... 31
2.3.1 What is parallel processing? ........................................................................... 312.3.2 Hardware architectures for parallel processing .............................................. 322.3.3 Approaches to parallel programming ............................................................. 362.3.4 Levels of parallelism.............................................................

In [11]:
metadata_info=[]
for i in range(len(compressed_docs)):    
    metadata_info.append('\''+str(compressed_docs[i].metadata)[23:])

In [12]:
metadata_info

["'cse\\\\semester8\\\\cloud_computing.pdf', 'page': 5}",
 "'cse\\\\semester8\\\\cloud_computing.pdf', 'page': 6}",
 "'cse\\\\semester8\\\\cloud_computing.pdf', 'page': 5}",
 "'cse\\\\semester8\\\\cloud_computing.pdf', 'page': 15}",
 "'cse\\\\semester8\\\\cloud_computing.pdf', 'page': 16}",
 "'cse\\\\semester8\\\\cloud_computing.pdf', 'page': 13}"]

In [13]:
llm = DeepInfra(model_id="meta-llama/Llama-2-7b-chat-hf")
llm.model_kwargs = {
    "temperature": 0.0,
    "repetition_penalty": 1.2,
    "max_new_tokens": 512,
    "top_p": 0.9,
}


In [14]:
context = "\n".join([f"{doc.page_content}\nMetadata: {doc.metadata}" for doc in compressed_docs])
response = llm( context=context,prompt=user_query)

In [15]:
response

"Parallel computing is a technique that involves dividing complex computational tasks into smaller sub-tasks that can be executed simultaneously on multiple processors or cores. The following are some of the key elements involved in parallel computing:\nProcessors/Cores: Parallel computing requires multiple processing units (CPUs) or cores to perform different parts of the computation simultaneously. These CPUs may be located on a single chip or distributed across several machines connected by a network.\nMemory Hierarchy: Each processor needs access to shared memory for data exchange between processes running concurrently. This hierarchy includes registers, cache memories, main memory, etc., which ensure efficient communication between processes without significant delays due to data transfer times over long distances within large systems with many nodes containing numerous processors each!\nInterconnects: Interconnects connect various nodes within a distributed system allowing them e